In [1]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

Installed autotime.py. To use it, type:
  %load_ext autotime


/Library/Python/2.7/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension(s)as a python packages.
  "as a python packages.", UserWarning)


In [2]:
import lxml

time: 3.19 ms


In [3]:
import xml.etree.cElementTree as ET
from lxml import etree
import re

time: 28.4 ms


In [1]:
topics = ['Top/Arts/Animation/.*']
print 'creating tree'
#
print 'created tree'

def regex_search(topic, topics):
    for x in topics:
        if re.search(x, topic) is not None:
            return True
    return False

def get_links(elem):
    if regex_search(elem.attrib.values()[0], topics):
        for x in elem.iterchildren(tag='link'):
            f.write(','.join([elem.values()[0], x.values()[0]])+'\n')
                    
def fast_iter(context, func):
    for event, elem in context:
        print 'gottem'
        get_links(elem)
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
        
    del context

with open('/Users/benkaufman/content.csv', 'a') as f:
    fast_iter(context, get_links)

creating tree
created tree


NameError: name 'context' is not defined

In [40]:
r = open('/Users/benkaufman/temp.txt', 'w')
r.close()

In [22]:
import rdflib

ImportError: No module named rdflib

In [48]:
%time
1+1

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 4.05 µs


2

In [55]:
import re

time: 879 µs


In [68]:
for x in topics:
    print x, re.search('Top/Arts/Animation/*',x)


Top/Arts/Animation <_sre.SRE_Match object at 0x103085ed0>
Top/Arts/Animation/Anime/Clubs_and_Organizations <_sre.SRE_Match object at 0x103085ed0>
time: 1.38 ms


In [38]:
topics = ['Top/Arts/Animation/']
context = etree.iterparse('/Users/benkaufman/Downloads/content.xml', events=('end',), tag='Topic')

def regex_search(topic, topics):
    for x in topics:
        if re.search(x, topic) is not None:
            return True
    return False

def get_links(elem):
    if regex_search(elem.attrib.values()[0], topics):
        for x in elem.iterchildren(tag='link'):
            f.write(','.join([elem.values()[0], x.values()[0]])+'\n')
                    
def fast_iter(context, func):
    for event, elem in context:
        func(elem)
        elem.clear()
        while elem.getprevious() is not None:
            del elem.getparent()[0]
    del context

def write_if_node(out, node):
    if node is not None:
        print node
        out.write(node)
        #out.write(etree.tostring(node, encoding='utf-8'))

def serialize_with_xpath(out, elem, xp1):
    '''Take our source <Record> element and apply two pre-compiled XPath classes.
    Return a node only if the first expression matches.
    '''
    #t = etree.SubElement(r, 'link')
    x = xp1(elem)
    if x:
        for el in x:
            out.write(el.get('{http://www.w3.org/TR/RDF/}resource') + '\n')


fi = open('/Users/benkaufman/content.csv', 'a')
xp1 = etree.XPath("//link")
#xp2 = etree.XPath("child::Copyright")
fast_iter(context, lambda elem: serialize_with_xpath(fi, elem, xp1))
fi.close()

time: 46.1 ms


In [73]:
fi = open('/Users/benkaufman/content.csv', 'r')
fi2 = open('/Users/benkaufman/content_processed.csv', 'wr')
max_depth = 0
lines = fi.readlines()
for line in lines:
    if line:
        category_string, _, website = line.partition('|')
        website = website.rstrip('\n')
        #engine.execute(text("insert into core.websites(website) values('{0}')".format(website)).execution_options(autocommit=True))
        depth = len(category_string.split('/'))
        if depth > max_depth:
            max_depth = depth
        
print max_depth
for line in lines:
    category_string, _, website = line.partition('|')
    website = website.rstrip('\n')
    levels = category_string.count('/')
    line_out = '|'.join(category_string.split('/')[1:])+''.join(['|' for x in range(0,(max_depth-levels)-1)])+'|'+website+'\n'
    fi2.write(line_out)
fi2.close()
fi.close()

#for line in fi.readlines():
#    num_pipes = line.count('|')

#engine.execute(text("insert into core.websites(website) values('{0}')".format(website).execution_options(autocommit=True))
print max_depth
table_string = "CREATE TEMPORARY TABLE dmoz_staging("
for col_ix in range(1,max_depth):
    print col_ix
    table_string += "cat_{0} varchar,".format(col_ix) 

table_string += "website varchar);"

print table_string
#engine.execute(table_string)

8
8
1
2
3
4
5
6
7
CREATE TEMPORARY TABLE dmoz_staging(cat_1 varchar,cat_2 varchar,cat_3 varchar,cat_4 varchar,cat_5 varchar,cat_6 varchar,cat_7 varchar,website varchar);


In [10]:
import sqlalchemy
from sqlalchemy import create_engine, text

In [9]:
engine = create_engine('postgresql://benkaufman@localhost/benkaufman')

In [11]:
engine.execute(text("insert into core.websites(website) values('http://www.cnn.com')").execution_options(autocommit=True))

In [13]:
engine.execute("create temporary table tempplz(col1 varchar)")

In [14]:
conn = engine.connect()

In [15]:
conn.execute("create temporary table tempplz(col1 varchar)")

ProgrammingError: (psycopg2.ProgrammingError) relation "tempplz" already exists
 [SQL: 'create temporary table tempplz(col1 varchar)']

In [16]:
conn.close()

In [59]:
import psycopg2
from sqlalchemy.exc import IntegrityError

In [61]:
try:
    engine.execute("""INSERT INTO core.site_categories(category, parent_id, site_id)
    SELECT d.cat_1, null, w.id
    FROM public.dmoz_staging d 
      JOIN core.websites w ON (w.website = d.website)
    GROUP BY 1,2,3;""")
    print 'INSERTED!'
except IntegrityError as e:
    print 'NOT INSERTED!'
else:
    raise
    


NOT INSERTED!


In [ ]:
### loop through queries